In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'

import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

In [2]:
root_path = r'/data/sunrui/celldata/20230724_HBEC_Yoko_Lib5_G+R-_DL/F0002/'
file_name = r'F0002.tif'

In [3]:
def show_anns(anns):
    if len(anns) == 0:
        return
    # 按面积大小对注释进行排序
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    img_shape = (sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1],1)


    
    # 创建一个空白图像
    # img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img = np.zeros(img_shape, dtype=np.uint8)
    img[:,:,0] = 0
    for i, ann in enumerate(sorted_anns):
        mask_thresholdU = 12000
        mask_thresholdD = 1000
        mask_area = ann['area']
        m = ann['segmentation']
        if(mask_area < mask_thresholdU and mask_area > mask_thresholdD):
            # gray_value = 2*i + 1
            gray_value = i + 1
            # color_mask = np.concatenate([np.random.random(3), [0.35]])
            img[m] = gray_value
        
        
        # print(img.shape)
    img = img
    
    return img

In [4]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(
    sam,
    pred_iou_thresh = 0.95,
    crop_overlap_ratio=0,
    min_mask_region_area = 1000,
    )

KeyboardInterrupt: 

In [ ]:
# image = cv2.imread('images/dog.jpg')
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
import skimage.io as skio
from tifffile import imread, imwrite

# #读取一系列图像
# raw_path = root_path#原始图像路径
# imgfiles = [os.path.join(raw_path, f) for f in os.listdir(raw_path) if f.endswith('.tif') or f.endswith('.tiff')]
# imgfiles.sort()

# img_raw = []
# print(len(imgfiles))
# for i in range(len(imgfiles)):
#     img_raw.append(skio.imread(imgfiles[i]).astype(np.uint16))
# print(len(img_raw))
# img_raw = np.array(img_raw)
# print(img_raw.shape)
# print(img_raw.shape[0])


img_raw = skio.imread(root_path + r'/PRE/test.tif',plugin="tifffile")
# img_raw = skio.imread(root_path + r'/PRE/test_seg.tif',plugin="tifffile")
print(img_raw.shape)
# img_raw = np.expand_dims(img_raw,axis=3)
# x = np.expand_dims(x,axis=3)
print(img_raw.dtype)

# #uint16转uint8
# max_val = np.max(img_raw)
# min_val = np.min(img_raw)

# # 归一化处理
# arr_normalized = (img_raw - min_val) * 255 / (max_val - min_val)

# # 将归一化后的数组转换为 uint8 类型
# arr_uint8 = arr_normalized.astype(np.uint8)

img = img_raw


(378, 1000, 1000, 1)
uint8


In [ ]:

image_folder =root_path + r'/01_GT/SAMSEG/'
if not os.path.exists(image_folder):
    os.makedirs(image_folder)

for i in range(img.shape[0]):
# for i in range(1):
    image = img[i]
    #uint16转换为uint8
    # uint16_img = image
    # uint16_img -= image.min()
    # uint16_img = uint16_img / (uint16_img.max()-uint16_img.min())
    # uint16_img *= 255
    # uint8_img = np.uint8(uint16_img)

    image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)

    # image = image.astype('uint8')
    # image = cv2.imread(r'/data/sunrui/celldata/20230414_hesc_Mixl1_yy/r03c03f01/01/t000.tif')
    # image = imread(r'/data/sunrui/celldata/20230414_hesc_Mixl1_yy/r03c03f01/01/t000.tif')

    # print(image.dtype)
    # print(image.shape)
    # print('-------------------------------------------------------')

    #生成masks
    masks = mask_generator.generate(image)
    mask_result = show_anns(masks)
    # print(mask_result.shape)

    # 图片保存的文件夹路径
    
    image_path = os.path.join(image_folder, f'man_seg{i:03d}.tif')
    imwrite(image_path,mask_result)

In [ ]:
print(mask_result.shape)
print(mask_result.dtype)

(1000, 1000, 1)
uint8
